# 井字棋 TicTacToe-v0

In [1]:
import sys
import math
import itertools
import collections
import logging

import numpy as np
import pandas as pd
import gym
import tensorflow as tf
from tensorflow import keras

import boardgame2
from boardgame2 import BLACK, WHITE

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG,
        format='%(asctime)s [%(levelname)s] %(message)s')

### 环境
环境在 boardgame2 里实现

In [2]:
env = gym.make('TicTacToe-v0')

### 智能体

In [3]:
def residual(x, filters, kernel_sizes=3, strides=1, activations='relu',
            regularizer=keras.regularizers.l2(1e-4)):
    shortcut = x
    for i, filte in enumerate(filters):
        kernel_size = kernel_sizes if isinstance(kernel_sizes, int) \
                else kernel_sizes[i]
        stride = strides if isinstance(strides, int) else strides[i]
        activation = activations if isinstance(activations, str) \
                else activations[i]
        z = keras.layers.Conv2D(filte, kernel_size, strides=stride,
                padding='same', kernel_regularizer=regularizer,
                bias_regularizer=regularizer)(x)
        y = keras.layers.BatchNormalization()(z)
        if i == len(filters) - 1:
            y = keras.layers.Add()([shortcut, y])
        x = keras.layers.Activation(activation)(y)
    return x

In [4]:
class AlphaZeroAgent:
    def __init__(self, env, batches=1, batch_size=4096,
            kwargs={}, load=None, sim_count=800,
            c_init=1.25, c_base=19652., prior_exploration_fraction=0.25):
        self.env = env
        self.board = np.zeros_like(env.board)
        self.batches = batches
        self.batch_size = batch_size

        self.net = self.build_network(**kwargs)
        self.reset_mcts()
        self.sim_count = sim_count # MCTS 次数
        self.c_init = c_init # PUCT 系数
        self.c_base = c_base # PUCT 系数
        self.prior_exploration_fraction = prior_exploration_fraction


    def build_network(self, conv_filters, residual_filters, policy_filters,
            learning_rate=0.001, regularizer=keras.regularizers.l2(1e-4)):
        # 公共部分
        inputs = keras.Input(shape=self.board.shape)
        x = keras.layers.Reshape(self.board.shape + (1,))(inputs)
        for conv_filter in conv_filters:
            z = keras.layers.Conv2D(conv_filter, 3, padding='same',
                    kernel_regularizer=regularizer,
                    bias_regularizer=regularizer)(x)
            y = keras.layers.BatchNormalization()(z)
            x = keras.layers.ReLU()(y)
        for residual_filter in residual_filters:
            x = residual(x, filters=residual_filter, regularizer=regularizer)
        intermediates = x

        # 概率部分
        for policy_filter in policy_filters:
            z = keras.layers.Conv2D(policy_filter, 3, padding='same',
                    kernel_regularizer=regularizer,
                    bias_regularizer=regularizer)(x)
            y = keras.layers.BatchNormalization()(z)
            x = keras.layers.ReLU()(y)
        logits = keras.layers.Conv2D(1, 3, padding='same',
                kernel_regularizer=regularizer, bias_regularizer=regularizer)(x)
        flattens = keras.layers.Flatten()(logits)
        softmaxs = keras.layers.Softmax()(flattens)
        probs = keras.layers.Reshape(self.board.shape)(softmaxs)

        # 价值部分
        z = keras.layers.Conv2D(1, 3, padding='same',
                kernel_regularizer=regularizer,
                bias_regularizer=regularizer)(intermediates)
        y = keras.layers.BatchNormalization()(z)
        x = keras.layers.ReLU()(y)
        flattens = keras.layers.Flatten()(x)
        vs = keras.layers.Dense(1, activation=keras.activations.tanh,
                kernel_regularizer=regularizer,
                bias_regularizer=regularizer)(flattens)

        model = keras.Model(inputs=inputs, outputs=[probs, vs])

        def categorical_crossentropy_2d(y_true, y_pred):
            labels = tf.reshape(y_true, [-1, self.board.size])
            preds = tf.reshape(y_pred, [-1, self.board.size])
            return keras.losses.categorical_crossentropy(labels, preds)

        loss = [categorical_crossentropy_2d, keras.losses.MSE]
        optimizer = keras.optimizers.Adam(learning_rate)
        model.compile(loss=loss, optimizer=optimizer)
        return model


    def reset_mcts(self):
        def zero_board_factory(): # 用于构造 default_dict
            return np.zeros_like(self.board, dtype=float)
        self.q = collections.defaultdict(zero_board_factory)
            # q值估计: board -> board
        self.count = collections.defaultdict(zero_board_factory)
            # q值计数: board -> board
        self.policy = {} # 策略: board -> board
        self.valid = {} # 有效位置: board -> board
        self.winner = {} # 赢家: board -> None or int


    def decide(self, observation, greedy=False, return_prob=False):
        # 计算策略
        board, player = observation
        canonical_board = player * board
        s = boardgame2.strfboard(canonical_board)
        while self.count[s].sum() < self.sim_count: # 多次 MCTS 搜索
            self.search(canonical_board, prior_noise=True)
        prob = self.count[s] / self.count[s].sum()

        # 采样
        location_index = np.random.choice(prob.size, p=prob.reshape(-1))
        location = np.unravel_index(location_index, prob.shape)
        if return_prob:
            return location, prob
        return location


    def learn(self, dfs):
        df = pd.concat(dfs).reset_index(drop=True)
        for batch in range(self.batches):
            indices = np.random.choice(len(df), size=self.batch_size)
            players, boards, probs, winners = (np.stack(
                    df.loc[indices, field]) for field in df.columns)
            canonical_boards = players[:, np.newaxis, np.newaxis] * boards
            vs = (players * winners)[:, np.newaxis]
            self.net.fit(canonical_boards, [probs, vs], verbose=0) # 训练
        self.reset_mcts()


    def search(self, board, prior_noise=False): # MCTS 搜索
        s = boardgame2.strfboard(board)

        if s not in self.winner:
            self.winner[s] = self.env.get_winner((board, BLACK)) # 计算赢家
        if self.winner[s] is not None: # 赢家确定的情况
            return self.winner[s]

        if s not in self.policy: # 未计算过策略的叶子节点
            pis, vs = self.net.predict(board[np.newaxis], verbose=0)
            pi, v = pis[0], vs[0]
            valid = self.env.get_valid((board, BLACK))
            masked_pi = pi * valid
            total_masked_pi = np.sum(masked_pi)
            if total_masked_pi <= 0: # 所有的有效动作都没有概率，偶尔可能发生
                masked_pi = valid # workaround
                total_masked_pi = np.sum(masked_pi)
            self.policy[s] = masked_pi / total_masked_pi
            self.valid[s] = valid
            return v

        # PUCT 上界计算
        count_sum = self.count[s].sum()
        coef = (self.c_init + np.log1p((1 + count_sum) / self.c_base)) * \
                math.sqrt(count_sum) / (1. + self.count[s])
        if prior_noise: # 先验噪声
            alpha = 1. / self.valid[s].sum()
            noise = np.random.gamma(alpha, 1., board.shape)
            noise *= self.valid[s]
            noise /= noise.sum()
            prior = (1. - self.prior_exploration_fraction) * \
                    self.policy[s] + \
                    self.prior_exploration_fraction * noise
        else:
            prior = self.policy[s]
        ub = np.where(self.valid[s], self.q[s] + coef * prior, np.nan)
        location_index = np.nanargmax(ub)
        location = np.unravel_index(location_index, board.shape)

        (next_board, next_player), _, _, _ = self.env.next_step(
                (board, BLACK), np.array(location))
        next_canonical_board = next_player * next_board
        next_v = self.search(next_canonical_board) # 递归搜索
        v = next_player * next_v

        self.count[s][location] += 1
        self.q[s][location] += (v - self.q[s][location]) / \
                self.count[s][location]
        return v

### 智能体和环境交互

In [5]:
def self_play(env, agent, return_trajectory=False, verbose=False):
    if return_trajectory:
        trajectory = []
    observation, _ = env.reset()
    for step in itertools.count():
        board, player = observation
        action, prob = agent.decide(observation, return_prob=True)
        if verbose:
            print(boardgame2.strfboard(board))
            logging.info('第 {} 步：玩家 {}, 动作 {}'.format(step, player,
                    action))
        observation, winner, terminated, truncated, _ = env.step(action)
        if return_trajectory:
            trajectory.append((player, board, prob))
        if terminated or truncated:
            if verbose:
                print(boardgame2.strfboard(observation[0]))
                logging.info('赢家 {}'.format(winner))
            break
    if return_trajectory:
        df_trajectory = pd.DataFrame(trajectory,
                columns=['player', 'board', 'prob'])
        df_trajectory['winner'] = winner
        return df_trajectory
    else:
        return winner

### 训练

In [6]:
"""
AlphaZero 参数，可用来求解比较大型的问题（如五子棋）
"""
train_iterations = 700000 # 训练迭代次数
train_episodes_per_iteration = 5000 # 每次迭代自我对弈回合数
batches = 10 # 每回合进行几次批学习
batch_size = 4096 # 批学习的批大小
sim_count = 800 # MCTS需要的计数
net_kwargs = {}
net_kwargs['conv_filters'] = [256,]
net_kwargs['residual_filters'] = [[256, 256],] * 19
net_kwargs['policy_filters'] = [256,]

"""
小规模参数，用来初步求解比较小的问题（如井字棋）
"""
train_iterations = 100
train_episodes_per_iteration = 100
batches = 2
batch_size = 64
sim_count = 200
net_kwargs = {}
net_kwargs['conv_filters'] = [256,]
net_kwargs['residual_filters'] = [[256, 256],]
net_kwargs['policy_filters'] = [256,]

agent = AlphaZeroAgent(env=env, kwargs=net_kwargs, sim_count=sim_count,
        batches=batches, batch_size=batch_size)


for iteration in range(train_iterations):
    # 自我对弈
    dfs_trajectory = []
    for episode in range(train_episodes_per_iteration):
        df_trajectory = self_play(env, agent,
                return_trajectory=True, verbose=False)
        logging.info('训练 {} 回合 {}: 收集到 {} 条经验'.format(
                iteration, episode, len(df_trajectory)))
        dfs_trajectory.append(df_trajectory)

    # 利用经验进行学习
    agent.learn(dfs_trajectory)
    logging.info('训练 {}: 学习完成'.format(iteration))

    # 演示训练结果
    self_play(env, agent, verbose=True)

2019-01-01 08:13:19,947 [INFO] 训练 0 回合 0: 收集到 7 条经验
2019-01-01 08:13:22,546 [INFO] 训练 0 回合 1: 收集到 5 条经验
2019-01-01 08:13:23,255 [INFO] 训练 0 回合 2: 收集到 5 条经验
2019-01-01 08:13:26,407 [INFO] 训练 0 回合 3: 收集到 9 条经验
2019-01-01 08:13:28,867 [INFO] 训练 0 回合 4: 收集到 9 条经验
2019-01-01 08:13:32,139 [INFO] 训练 0 回合 5: 收集到 9 条经验
2019-01-01 08:13:34,535 [INFO] 训练 0 回合 6: 收集到 6 条经验
2019-01-01 08:13:37,791 [INFO] 训练 0 回合 7: 收集到 9 条经验
2019-01-01 08:13:38,733 [INFO] 训练 0 回合 8: 收集到 7 条经验
2019-01-01 08:13:39,935 [INFO] 训练 0 回合 9: 收集到 7 条经验
2019-01-01 08:13:42,072 [INFO] 训练 0 回合 10: 收集到 8 条经验
2019-01-01 08:13:44,203 [INFO] 训练 0 回合 11: 收集到 9 条经验
2019-01-01 08:13:46,556 [INFO] 训练 0 回合 12: 收集到 8 条经验
2019-01-01 08:13:48,737 [INFO] 训练 0 回合 13: 收集到 7 条经验
2019-01-01 08:13:50,935 [INFO] 训练 0 回合 14: 收集到 9 条经验
2019-01-01 08:13:52,076 [INFO] 训练 0 回合 15: 收集到 7 条经验
2019-01-01 08:13:52,129 [INFO] 训练 0 回合 16: 收集到 7 条经验
2019-01-01 08:13:55,166 [INFO] 训练 0 回合 17: 收集到 7 条经验
2019-01-01 08:13:55,722 [INFO] 训练 0 回合 18: 收集到 5 条经验
201

2019-01-01 08:18:10,356 [INFO] 训练 2 回合 66: 收集到 5 条经验
2019-01-01 08:18:10,946 [INFO] 训练 2 回合 67: 收集到 6 条经验
2019-01-01 08:18:11,038 [INFO] 训练 2 回合 68: 收集到 6 条经验
2019-01-01 08:18:11,669 [INFO] 训练 2 回合 69: 收集到 7 条经验
2019-01-01 08:18:12,140 [INFO] 训练 2 回合 70: 收集到 7 条经验
2019-01-01 08:18:12,451 [INFO] 训练 2 回合 71: 收集到 7 条经验
2019-01-01 08:18:13,409 [INFO] 训练 2 回合 72: 收集到 7 条经验
2019-01-01 08:18:14,531 [INFO] 训练 2 回合 73: 收集到 9 条经验
2019-01-01 08:18:14,765 [INFO] 训练 2 回合 74: 收集到 5 条经验
2019-01-01 08:18:15,372 [INFO] 训练 2 回合 75: 收集到 5 条经验
2019-01-01 08:18:16,895 [INFO] 训练 2 回合 76: 收集到 9 条经验
2019-01-01 08:18:17,520 [INFO] 训练 2 回合 77: 收集到 9 条经验
2019-01-01 08:18:19,385 [INFO] 训练 2 回合 78: 收集到 9 条经验
2019-01-01 08:18:20,707 [INFO] 训练 2 回合 79: 收集到 8 条经验
2019-01-01 08:18:20,720 [INFO] 训练 2 回合 80: 收集到 9 条经验
2019-01-01 08:18:21,909 [INFO] 训练 2 回合 81: 收集到 9 条经验
2019-01-01 08:18:22,171 [INFO] 训练 2 回合 82: 收集到 7 条经验
2019-01-01 08:18:23,613 [INFO] 训练 2 回合 83: 收集到 9 条经验
2019-01-01 08:18:23,832 [INFO] 训练 2 回合 84: 收集到

oxo
+x+
x+o
2019-01-01 08:20:26,536 [INFO] 第 6 步：玩家 1, 动作 (1, 2)
oxo
+xo
x+o
2019-01-01 08:20:26,536 [INFO] 赢家 1
2019-01-01 08:20:27,521 [INFO] 训练 4 回合 0: 收集到 9 条经验
2019-01-01 08:20:30,046 [INFO] 训练 4 回合 1: 收集到 6 条经验
2019-01-01 08:20:32,895 [INFO] 训练 4 回合 2: 收集到 7 条经验
2019-01-01 08:20:35,708 [INFO] 训练 4 回合 3: 收集到 5 条经验
2019-01-01 08:20:38,200 [INFO] 训练 4 回合 4: 收集到 9 条经验
2019-01-01 08:20:41,402 [INFO] 训练 4 回合 5: 收集到 7 条经验
2019-01-01 08:20:44,594 [INFO] 训练 4 回合 6: 收集到 7 条经验
2019-01-01 08:20:46,217 [INFO] 训练 4 回合 7: 收集到 6 条经验
2019-01-01 08:20:46,814 [INFO] 训练 4 回合 8: 收集到 5 条经验
2019-01-01 08:20:47,358 [INFO] 训练 4 回合 9: 收集到 8 条经验
2019-01-01 08:20:51,034 [INFO] 训练 4 回合 10: 收集到 9 条经验
2019-01-01 08:20:53,106 [INFO] 训练 4 回合 11: 收集到 7 条经验
2019-01-01 08:20:53,567 [INFO] 训练 4 回合 12: 收集到 7 条经验
2019-01-01 08:20:55,229 [INFO] 训练 4 回合 13: 收集到 9 条经验
2019-01-01 08:20:55,886 [INFO] 训练 4 回合 14: 收集到 9 条经验
2019-01-01 08:20:56,717 [INFO] 训练 4 回合 15: 收集到 9 条经验
2019-01-01 08:20:57,434 [INFO] 训练 4 回合 16: 收集到 7 

2019-01-01 08:23:19,001 [INFO] 训练 5 回合 42: 收集到 7 条经验
2019-01-01 08:23:20,550 [INFO] 训练 5 回合 43: 收集到 9 条经验
2019-01-01 08:23:21,125 [INFO] 训练 5 回合 44: 收集到 5 条经验
2019-01-01 08:23:23,558 [INFO] 训练 5 回合 45: 收集到 9 条经验
2019-01-01 08:23:23,781 [INFO] 训练 5 回合 46: 收集到 5 条经验
2019-01-01 08:23:25,251 [INFO] 训练 5 回合 47: 收集到 9 条经验
2019-01-01 08:23:27,081 [INFO] 训练 5 回合 48: 收集到 9 条经验
2019-01-01 08:23:28,133 [INFO] 训练 5 回合 49: 收集到 7 条经验
2019-01-01 08:23:29,347 [INFO] 训练 5 回合 50: 收集到 9 条经验
2019-01-01 08:23:29,431 [INFO] 训练 5 回合 51: 收集到 5 条经验
2019-01-01 08:23:30,515 [INFO] 训练 5 回合 52: 收集到 9 条经验
2019-01-01 08:23:31,563 [INFO] 训练 5 回合 53: 收集到 9 条经验
2019-01-01 08:23:32,118 [INFO] 训练 5 回合 54: 收集到 7 条经验
2019-01-01 08:23:32,323 [INFO] 训练 5 回合 55: 收集到 8 条经验
2019-01-01 08:23:33,765 [INFO] 训练 5 回合 56: 收集到 9 条经验
2019-01-01 08:23:35,735 [INFO] 训练 5 回合 57: 收集到 9 条经验
2019-01-01 08:23:36,260 [INFO] 训练 5 回合 58: 收集到 7 条经验
2019-01-01 08:23:37,246 [INFO] 训练 5 回合 59: 收集到 9 条经验
2019-01-01 08:23:38,763 [INFO] 训练 5 回合 60: 收集到

2019-01-01 08:25:47,313 [INFO] 训练 6 回合 89: 收集到 7 条经验
2019-01-01 08:25:47,968 [INFO] 训练 6 回合 90: 收集到 7 条经验
2019-01-01 08:25:48,905 [INFO] 训练 6 回合 91: 收集到 6 条经验
2019-01-01 08:25:48,953 [INFO] 训练 6 回合 92: 收集到 7 条经验
2019-01-01 08:25:50,431 [INFO] 训练 6 回合 93: 收集到 9 条经验
2019-01-01 08:25:50,625 [INFO] 训练 6 回合 94: 收集到 5 条经验
2019-01-01 08:25:50,769 [INFO] 训练 6 回合 95: 收集到 5 条经验
2019-01-01 08:25:50,934 [INFO] 训练 6 回合 96: 收集到 5 条经验
2019-01-01 08:25:50,984 [INFO] 训练 6 回合 97: 收集到 5 条经验
2019-01-01 08:25:51,286 [INFO] 训练 6 回合 98: 收集到 7 条经验
2019-01-01 08:25:51,877 [INFO] 训练 6 回合 99: 收集到 7 条经验
2019-01-01 08:25:52,779 [INFO] 训练 6: 学习完成
+++
+++
+++
2019-01-01 08:25:54,251 [INFO] 第 0 步：玩家 1, 动作 (0, 0)
o++
+++
+++
2019-01-01 08:25:55,482 [INFO] 第 1 步：玩家 -1, 动作 (2, 0)
o++
+++
x++
2019-01-01 08:25:56,595 [INFO] 第 2 步：玩家 1, 动作 (0, 2)
o+o
+++
x++
2019-01-01 08:25:56,758 [INFO] 第 3 步：玩家 -1, 动作 (1, 0)
o+o
x++
x++
2019-01-01 08:25:56,979 [INFO] 第 4 步：玩家 1, 动作 (0, 1)
ooo
x++
x++
2019-01-01 08:25:56,979 [INFO] 赢家 1


...

2019-01-01 10:47:46,149 [INFO] 训练 93 回合 91: 收集到 5 条经验
2019-01-01 10:47:46,230 [INFO] 训练 93 回合 92: 收集到 8 条经验
2019-01-01 10:47:47,365 [INFO] 训练 93 回合 93: 收集到 9 条经验
2019-01-01 10:47:47,533 [INFO] 训练 93 回合 94: 收集到 7 条经验
2019-01-01 10:47:47,635 [INFO] 训练 93 回合 95: 收集到 9 条经验
2019-01-01 10:47:47,678 [INFO] 训练 93 回合 96: 收集到 5 条经验
2019-01-01 10:47:48,360 [INFO] 训练 93 回合 97: 收集到 7 条经验
2019-01-01 10:47:48,370 [INFO] 训练 93 回合 98: 收集到 9 条经验
2019-01-01 10:47:48,381 [INFO] 训练 93 回合 99: 收集到 7 条经验
2019-01-01 10:47:49,320 [INFO] 训练 93: 学习完成
+++
+++
+++
2019-01-01 10:47:50,878 [INFO] 第 0 步：玩家 1, 动作 (2, 0)
+++
+++
o++
2019-01-01 10:47:52,195 [INFO] 第 1 步：玩家 -1, 动作 (0, 0)
x++
+++
o++
2019-01-01 10:47:52,936 [INFO] 第 2 步：玩家 1, 动作 (0, 2)
x+o
+++
o++
2019-01-01 10:47:53,021 [INFO] 第 3 步：玩家 -1, 动作 (1, 1)
x+o
+x+
o++
2019-01-01 10:47:53,198 [INFO] 第 4 步：玩家 1, 动作 (0, 1)
xoo
+x+
o++
2019-01-01 10:47:53,396 [INFO] 第 5 步：玩家 -1, 动作 (2, 2)
xoo
+x+
o+x
2019-01-01 10:47:53,396 [INFO] 赢家 -1
2019-01-01 10:47:57,361 [INFO

2019-01-01 10:50:04,529 [INFO] 训练 95 回合 24: 收集到 7 条经验
2019-01-01 10:50:04,539 [INFO] 训练 95 回合 25: 收集到 9 条经验
2019-01-01 10:50:04,936 [INFO] 训练 95 回合 26: 收集到 8 条经验
2019-01-01 10:50:05,511 [INFO] 训练 95 回合 27: 收集到 5 条经验
2019-01-01 10:50:06,474 [INFO] 训练 95 回合 28: 收集到 7 条经验
2019-01-01 10:50:07,029 [INFO] 训练 95 回合 29: 收集到 9 条经验
2019-01-01 10:50:08,250 [INFO] 训练 95 回合 30: 收集到 7 条经验
2019-01-01 10:50:09,789 [INFO] 训练 95 回合 31: 收集到 5 条经验
2019-01-01 10:50:11,039 [INFO] 训练 95 回合 32: 收集到 9 条经验
2019-01-01 10:50:11,683 [INFO] 训练 95 回合 33: 收集到 7 条经验
2019-01-01 10:50:13,050 [INFO] 训练 95 回合 34: 收集到 9 条经验
2019-01-01 10:50:13,955 [INFO] 训练 95 回合 35: 收集到 9 条经验
2019-01-01 10:50:14,806 [INFO] 训练 95 回合 36: 收集到 9 条经验
2019-01-01 10:50:15,214 [INFO] 训练 95 回合 37: 收集到 9 条经验
2019-01-01 10:50:17,611 [INFO] 训练 95 回合 38: 收集到 9 条经验
2019-01-01 10:50:18,664 [INFO] 训练 95 回合 39: 收集到 7 条经验
2019-01-01 10:50:19,466 [INFO] 训练 95 回合 40: 收集到 7 条经验
2019-01-01 10:50:19,476 [INFO] 训练 95 回合 41: 收集到 9 条经验
2019-01-01 10:50:21,243 [INF

2019-01-01 10:52:03,036 [INFO] 训练 96 回合 69: 收集到 9 条经验
2019-01-01 10:52:03,227 [INFO] 训练 96 回合 70: 收集到 7 条经验
2019-01-01 10:52:03,227 [INFO] 训练 96 回合 71: 收集到 5 条经验
2019-01-01 10:52:03,404 [INFO] 训练 96 回合 72: 收集到 5 条经验
2019-01-01 10:52:03,488 [INFO] 训练 96 回合 73: 收集到 5 条经验
2019-01-01 10:52:03,544 [INFO] 训练 96 回合 74: 收集到 5 条经验
2019-01-01 10:52:04,135 [INFO] 训练 96 回合 75: 收集到 9 条经验
2019-01-01 10:52:04,383 [INFO] 训练 96 回合 76: 收集到 9 条经验
2019-01-01 10:52:05,072 [INFO] 训练 96 回合 77: 收集到 5 条经验
2019-01-01 10:52:05,323 [INFO] 训练 96 回合 78: 收集到 9 条经验
2019-01-01 10:52:05,595 [INFO] 训练 96 回合 79: 收集到 7 条经验
2019-01-01 10:52:06,391 [INFO] 训练 96 回合 80: 收集到 7 条经验
2019-01-01 10:52:06,613 [INFO] 训练 96 回合 81: 收集到 5 条经验
2019-01-01 10:52:06,787 [INFO] 训练 96 回合 82: 收集到 5 条经验
2019-01-01 10:52:07,591 [INFO] 训练 96 回合 83: 收集到 7 条经验
2019-01-01 10:52:07,854 [INFO] 训练 96 回合 84: 收集到 5 条经验
2019-01-01 10:52:08,348 [INFO] 训练 96 回合 85: 收集到 9 条经验
2019-01-01 10:52:08,957 [INFO] 训练 96 回合 86: 收集到 7 条经验
2019-01-01 10:52:09,061 [INF

2019-01-01 10:53:45,649 [INFO] 训练 98 回合 4: 收集到 9 条经验
2019-01-01 10:53:46,717 [INFO] 训练 98 回合 5: 收集到 9 条经验
2019-01-01 10:53:49,477 [INFO] 训练 98 回合 6: 收集到 9 条经验
2019-01-01 10:53:51,376 [INFO] 训练 98 回合 7: 收集到 5 条经验
2019-01-01 10:53:53,153 [INFO] 训练 98 回合 8: 收集到 7 条经验
2019-01-01 10:53:56,799 [INFO] 训练 98 回合 9: 收集到 9 条经验
2019-01-01 10:53:57,586 [INFO] 训练 98 回合 10: 收集到 9 条经验
2019-01-01 10:53:59,880 [INFO] 训练 98 回合 11: 收集到 9 条经验
2019-01-01 10:54:02,266 [INFO] 训练 98 回合 12: 收集到 9 条经验
2019-01-01 10:54:03,030 [INFO] 训练 98 回合 13: 收集到 5 条经验
2019-01-01 10:54:03,581 [INFO] 训练 98 回合 14: 收集到 9 条经验
2019-01-01 10:54:04,491 [INFO] 训练 98 回合 15: 收集到 7 条经验
2019-01-01 10:54:05,219 [INFO] 训练 98 回合 16: 收集到 5 条经验
2019-01-01 10:54:06,119 [INFO] 训练 98 回合 17: 收集到 9 条经验
2019-01-01 10:54:06,822 [INFO] 训练 98 回合 18: 收集到 7 条经验
2019-01-01 10:54:07,195 [INFO] 训练 98 回合 19: 收集到 7 条经验
2019-01-01 10:54:07,771 [INFO] 训练 98 回合 20: 收集到 7 条经验
2019-01-01 10:54:08,088 [INFO] 训练 98 回合 21: 收集到 5 条经验
2019-01-01 10:54:08,249 [INFO] 训练 

2019-01-01 10:55:46,814 [INFO] 训练 99 回合 44: 收集到 5 条经验
2019-01-01 10:55:47,947 [INFO] 训练 99 回合 45: 收集到 5 条经验
2019-01-01 10:55:48,770 [INFO] 训练 99 回合 46: 收集到 9 条经验
2019-01-01 10:55:49,338 [INFO] 训练 99 回合 47: 收集到 5 条经验
2019-01-01 10:55:50,799 [INFO] 训练 99 回合 48: 收集到 9 条经验
2019-01-01 10:55:51,030 [INFO] 训练 99 回合 49: 收集到 5 条经验
2019-01-01 10:55:51,106 [INFO] 训练 99 回合 50: 收集到 6 条经验
2019-01-01 10:55:51,251 [INFO] 训练 99 回合 51: 收集到 9 条经验
2019-01-01 10:55:51,910 [INFO] 训练 99 回合 52: 收集到 9 条经验
2019-01-01 10:55:51,982 [INFO] 训练 99 回合 53: 收集到 7 条经验
2019-01-01 10:55:52,027 [INFO] 训练 99 回合 54: 收集到 6 条经验
2019-01-01 10:55:52,148 [INFO] 训练 99 回合 55: 收集到 6 条经验
2019-01-01 10:55:52,187 [INFO] 训练 99 回合 56: 收集到 7 条经验
2019-01-01 10:55:53,063 [INFO] 训练 99 回合 57: 收集到 5 条经验
2019-01-01 10:55:53,570 [INFO] 训练 99 回合 58: 收集到 5 条经验
2019-01-01 10:55:54,065 [INFO] 训练 99 回合 59: 收集到 9 条经验
2019-01-01 10:55:54,606 [INFO] 训练 99 回合 60: 收集到 9 条经验
2019-01-01 10:55:55,017 [INFO] 训练 99 回合 61: 收集到 9 条经验
2019-01-01 10:55:56,710 [INF

### 演示训练结果

In [7]:
self_play(env, agent, verbose=True)

+++
+++
+++
2019-01-01 10:56:20,926 [INFO] 第 0 步：玩家 1, 动作 (0, 1)
+o+
+++
+++
2019-01-01 10:56:22,143 [INFO] 第 1 步：玩家 -1, 动作 (0, 0)
xo+
+++
+++
2019-01-01 10:56:23,196 [INFO] 第 2 步：玩家 1, 动作 (2, 0)
xo+
+++
o++
2019-01-01 10:56:23,713 [INFO] 第 3 步：玩家 -1, 动作 (1, 1)
xo+
+x+
o++
2019-01-01 10:56:23,920 [INFO] 第 4 步：玩家 1, 动作 (2, 2)
xo+
+x+
o+o
2019-01-01 10:56:23,953 [INFO] 第 5 步：玩家 -1, 动作 (2, 1)
xo+
+x+
oxo
2019-01-01 10:56:23,999 [INFO] 第 6 步：玩家 1, 动作 (0, 2)
xoo
+x+
oxo
2019-01-01 10:56:24,144 [INFO] 第 7 步：玩家 -1, 动作 (1, 2)
xoo
+xx
oxo
2019-01-01 10:56:24,154 [INFO] 第 8 步：玩家 1, 动作 (1, 0)
xoo
oxx
oxo
2019-01-01 10:56:24,154 [INFO] 赢家 0


0